In [27]:
import pandas as pd

PLT_SHOW = plt.show()
weather_set = pd.read_csv("CSV/Weather_losAngeles.csv", sep=",", index_col="timestamp")

In [28]:
weather_set.head()

,city,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
timestamp,,,,,,,,
2015-01-01 08:53:00,Burbank,3.0,33.0,Fair,989.11,0.0,0.0,3.0
2015-01-01 09:53:00,Burbank,2.0,33.0,Fair,989.77,0.0,0.0,2.0
2015-01-01 10:53:00,Burbank,2.0,33.0,Fair,989.44,0.0,0.0,2.0
2015-01-01 11:53:00,Burbank,1.0,33.0,Fair,989.77,0.0,0.0,1.0
2015-01-01 12:53:00,Burbank,1.0,33.0,Fair,990.10,0.0,0.0,1.0


In [29]:
weather_set.columns

Index(['city', 'temperature', 'cloud_cover', 'cloud_cover_description',
       'pressure', 'windspeed', 'precipitation', 'felt_temperature'],
      dtype='object')

In [30]:
weather_set.describe()

,temperature,cloud_cover,pressure,windspeed,precipitation,felt_temperature
count,77260.000000,77281.000000,77335.000000,77151.000000,77362.000000,77254.000000
mean,18.405177,30.304214,986.871789,8.452165,0.044915,18.249579
std,6.564116,5.243314,3.650721,6.813668,0.398354,6.418661
min,-17.000000,4.000000,968.370000,0.000000,0.000000,-24.000000
25%,14.000000,27.000000,984.500000,0.000000,0.000000,14.000000
50%,18.000000,33.000000,986.480000,7.000000,0.000000,18.000000
75%,22.000000,34.000000,989.110000,13.000000,0.000000,22.000000
max,54.000000,47.000000,1002.940000,65.000000,18.540000,44.000000


In [31]:
weather_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77362 entries, 2015-01-01 08:53:00 to 2023-01-01 07:55:00
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   city                     77362 non-null  object 
 1   temperature              77260 non-null  float64
 2   cloud_cover              77281 non-null  float64
 3   cloud_cover_description  77281 non-null  object 
 4   pressure                 77335 non-null  float64
 5   windspeed                77151 non-null  float64
 6   precipitation            77362 non-null  float64
 7   felt_temperature         77254 non-null  float64
dtypes: float64(6), object(2)
memory usage: 5.3+ MB


In [32]:
weather_set.isnull()

,city,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
timestamp,,,,,,,,
2015-01-01 08:53:00,False,False,False,False,False,False,False,False
2015-01-01 09:53:00,False,False,False,False,False,False,False,False
2015-01-01 10:53:00,False,False,False,False,False,False,False,False
2015-01-01 11:53:00,False,False,False,False,False,False,False,False
2015-01-01 12:53:00,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
2023-01-01 06:36:00,False,False,False,False,False,False,False,False
2023-01-01 06:53:00,False,False,False,False,False,False,False,False
2023-01-01 07:18:00,False,False,False,False,False,False,False,False


In [33]:
weather_set.dropna(axis=0, inplace=False)

,city,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
timestamp,,,,,,,,
2015-01-01 08:53:00,Burbank,3.0,33.0,Fair,989.11,0.0,0.00,3.0
2015-01-01 09:53:00,Burbank,2.0,33.0,Fair,989.77,0.0,0.00,2.0
2015-01-01 10:53:00,Burbank,2.0,33.0,Fair,989.44,0.0,0.00,2.0
2015-01-01 11:53:00,Burbank,1.0,33.0,Fair,989.77,0.0,0.00,1.0
2015-01-01 12:53:00,Burbank,1.0,33.0,Fair,990.10,0.0,0.00,1.0
...,...,...,...,...,...,...,...,...
2023-01-01 06:36:00,Burbank,13.0,12.0,Rain,979.24,30.0,2.03,13.0
2023-01-01 06:53:00,Burbank,13.0,12.0,Rain / Windy,978.91,33.0,2.79,13.0
2023-01-01 07:18:00,Burbank,13.0,40.0,Heavy Rain,978.25,31.0,1.27,13.0


In [34]:
len(weather_set) - len(weather_set.dropna())

416

In [35]:
clean_weather_set = weather_set.dropna(axis=0)
clean_weather_set

,city,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
timestamp,,,,,,,,
2015-01-01 08:53:00,Burbank,3.0,33.0,Fair,989.11,0.0,0.00,3.0
2015-01-01 09:53:00,Burbank,2.0,33.0,Fair,989.77,0.0,0.00,2.0
2015-01-01 10:53:00,Burbank,2.0,33.0,Fair,989.44,0.0,0.00,2.0
2015-01-01 11:53:00,Burbank,1.0,33.0,Fair,989.77,0.0,0.00,1.0
2015-01-01 12:53:00,Burbank,1.0,33.0,Fair,990.10,0.0,0.00,1.0
...,...,...,...,...,...,...,...,...
2023-01-01 06:36:00,Burbank,13.0,12.0,Rain,979.24,30.0,2.03,13.0
2023-01-01 06:53:00,Burbank,13.0,12.0,Rain / Windy,978.91,33.0,2.79,13.0
2023-01-01 07:18:00,Burbank,13.0,40.0,Heavy Rain,978.25,31.0,1.27,13.0


In [36]:
print("weather_set:",len(weather_set))
print("clean_weather_set:",len(clean_weather_set))
print("Datasets with null values:",len(weather_set) - len(weather_set.dropna()))

weather_set: 77362
clean_weather_set: 76946
Datasets with null values: 416


In [37]:
weather_set[weather_set.isnull().any(axis=1)]

,city,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
timestamp,,,,,,,,
2015-01-26 17:53:00,Burbank,NaN,28.0,Mostly Cloudy,988.78,9.0,0.00,NaN
2015-05-27 17:51:00,Burbank,17.0,NaN,NaN,987.80,7.0,0.00,17.0
2015-05-27 17:53:00,Burbank,17.0,NaN,NaN,987.80,11.0,0.00,17.0
2015-05-27 18:53:00,Burbank,19.0,34.0,Fair,987.47,NaN,0.00,19.0
2015-08-03 16:53:00,Burbank,25.0,NaN,NaN,985.16,6.0,0.00,26.0
...,...,...,...,...,...,...,...,...
2022-11-08 12:53:00,Burbank,NaN,40.0,Heavy Rain,982.86,13.0,4.83,NaN
2022-11-16 12:53:00,Burbank,18.0,33.0,Fair,993.72,NaN,0.00,18.0
2022-11-20 21:53:00,Burbank,23.0,34.0,Fair,991.09,NaN,0.00,23.0


In [38]:
metro_set = pd.read_csv("CSV/metro_2017_2022.csv", sep=",", index_col="transaction_id")

In [39]:
metro_set.columns

Index(['start_time', 'end_time', 'start_station_id', 'start_station_lat',
       'start_station_lon', 'end_station_id', 'end_station_lat',
       'end_station_lon', 'bike_id'],
      dtype='object')

In [40]:
metro_set.describe()

,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon
count,1.552782e+06,1.542572e+06,1.542572e+06,1.552782e+06,1.513071e+06,1.513070e+06
mean,3.615318e+03,3.404273e+01,-1.182708e+02,3.602597e+03,3.404177e+01,-1.182709e+02
std,6.527251e+02,3.053914e-01,2.180630e+00,6.507752e+02,4.389338e-01,2.166677e+00
min,3.000000e+03,3.371098e+01,-1.184954e+02,3.000000e+03,-9.000000e+01,-1.184954e+02
25%,3.032000e+03,3.402765e+01,-1.183056e+02,3.031000e+03,3.402762e+01,-1.183056e+02
50%,3.074000e+03,3.404613e+01,-1.182587e+02,3.069000e+03,3.404607e+01,-1.182587e+02
75%,4.250000e+03,3.405194e+01,-1.182483e+02,4.248000e+03,3.405110e+01,-1.182483e+02
max,4.638000e+03,5.570553e+01,1.182383e+02,4.640000e+03,5.570553e+01,3.760654e+01


In [41]:
metro_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1552782 entries, 0 to 1552781
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   start_time         1552782 non-null  object 
 1   end_time           1552782 non-null  object 
 2   start_station_id   1552782 non-null  int64  
 3   start_station_lat  1542572 non-null  float64
 4   start_station_lon  1542572 non-null  float64
 5   end_station_id     1552782 non-null  int64  
 6   end_station_lat    1513071 non-null  float64
 7   end_station_lon    1513070 non-null  float64
 8   bike_id            1552782 non-null  object 
dtypes: float64(4), int64(2), object(3)
memory usage: 118.5+ MB


In [42]:
clean_metro_set = metro_set.dropna(axis=0)
clean_metro_set

,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
transaction_id,,,,,,,,,
0,16/45/00 29-05-2018,17/08/00 29-05-2018,3068,34.053200,-118.250954,3014,34.056610,-118.237213,5772
1,18/34/00 09-08-2018,19/09/00 09-08-2018,4126,33.710979,-118.284668,4126,33.710979,-118.284668,12003
2,13/37/00 01-09-2021,13/48/00 01-09-2021,3082,34.046520,-118.237411,3031,34.044701,-118.252441,6107
3,12/40/00 31-08-2018,13/35/00 31-08-2018,4215,34.014309,-118.491341,4210,33.984341,-118.471550,6474
4,16/41/00 29-03-2018,17/15/00 29-03-2018,4126,33.710979,-118.284668,4126,33.710979,-118.284668,6374
...,...,...,...,...,...,...,...,...,...
1552776,13/32/00 30-06-2018,13/54/00 30-06-2018,3016,34.052898,-118.241562,3035,34.048401,-118.260948,6705
1552778,13/34/00 26-10-2018,13/39/00 26-10-2018,3038,34.046822,-118.248352,3030,34.051941,-118.243530,12334
1552779,13/38/00 04-10-2018,15/51/00 04-10-2018,4212,33.988129,-118.471741,4202,33.991161,-118.468292,6276


In [43]:
print("metro_set:",len(metro_set))
print("clean_metro_set:",len(clean_metro_set))
print("Datasets with null values:",len(metro_set) - len(metro_set.dropna()))

metro_set: 1552782
clean_metro_set: 1507366
Datasets with null values: 45416


In [44]:
metro_set[metro_set.isnull().any(axis=1)]

,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
transaction_id,,,,,,,,,
19,22/19/00 12-07-2020,22/49/00 12-07-2020,4285,NaN,NaN,4285,NaN,NaN,15860
48,17/17/00 19-01-2020,17/23/00 19-01-2020,4285,NaN,NaN,4352,33.988419,-118.451630,15607
95,17/16/15 13-10-2019,17/23/01 13-10-2019,4285,NaN,NaN,4346,33.984341,-118.471550,15392
115,18/48/00 28-05-2020,19/59/00 28-05-2020,4285,NaN,NaN,4346,33.984341,-118.471550,15135
189,18/50/00 24-08-2019,19/37/00 24-08-2019,4286,NaN,NaN,4285,NaN,NaN,15289
...,...,...,...,...,...,...,...,...,...
1552671,18/03/00 12-07-2021,18/04/00 12-07-2021,4567,34.020180,-118.404030,3000,NaN,NaN,15544
1552703,13/51/00 15-05-2019,14/04/00 15-05-2019,4286,NaN,NaN,4286,NaN,NaN,16178
1552704,18/46/00 28-10-2022,18/51/00 28-10-2022,4450,34.063492,-118.287163,3000,NaN,NaN,15520


In [45]:
weather_groups_agg = clean_weather_set.groupby("cloud_cover_description").agg(['mean', 'max', 'count'])
weather_groups_agg

C:\Users\yusuf\AppData\Local\Temp\ipykernel_5384\976572699.py:1: FutureWarning: ['city'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  weather_groups_agg = clean_weather_set.groupby("cloud_cover_description").agg(['mean', 'max', 'count'])


temperature              cloud_cover               \
                               mean   max  count        mean   max  count   
cloud_cover_description                                                     
Blowing Dust              17.400000  21.0     10   19.000000  19.0     10   
Blowing Dust / Windy      14.777778  16.0     18   19.000000  19.0     18   
Cloudy                    15.534459  39.0  11782   26.000000  26.0  11782   
Cloudy / Windy            15.818182  25.0     11   26.000000  26.0     11   
Fair                      19.924586  54.0  47140   33.540878  34.0  47140   
Fair / Windy              18.342723  30.0    213   33.596244  34.0    213   
Fog                       12.114929  19.0    844   20.000000  20.0    844   
Haze                      17.418913  37.0   2337   21.000000  21.0   2337   
Haze / Windy              19.000000  19.0      2   21.000000  21.0      2   
Heavy Rain                11.275556  21.0    225   40.000000  40.0    225   
Heavy Rain / Windy        12.000000  16.0     17   40.000000  40.0     17   
Heavy T-Storm             12.200000  13.0      5    4.000000   4.0      5   
Light Rain                12.790972  41.0   2038   11.000000  11.0   2038   
Light Rain / Windy        12.625000  16.0     24   11.000000  11.0     24   
Light Rain with Thunder   21.000000  26.0      6    4.000000   4.0      6   
Mostly Cloudy             16.027015  39.0   4664   27.476415  28.0   4664   
Mostly Cloudy / Windy     17.000000  25.0     14   27.285714  28.0     14   
Partly Cloudy             17.438710  42.0   6355   29.483871  30.0   6355   
Partly Cloudy / Windy     19.000000  37.0     25   29.480000  30.0     25   
Rain                      11.794000  31.0    500   12.000000  12.0    500   
Rain / Windy              11.466667  14.0     15   12.000000  12.0     15   
Smoke                     23.086811  38.0    599   22.000000  22.0    599   
Smoke / Windy             17.666667  24.0     21   22.000000  22.0     21   
T-Storm                   16.035714  37.0     28    4.000000   4.0     28   
Thunder                   22.875000  31.0      8   38.000000  38.0      8   
Thunder in the Vicinity   22.605263  42.0     38   39.894737  47.0     38   
Widespread Dust           35.000000  38.0      7   19.000000  19.0      7   

                           pressure                  windspeed               \
                               mean      max  count       mean   max  count   
cloud_cover_description                                                       
Blowing Dust             984.405000   988.13     10  20.600000  31.0     10   
Blowing Dust / Windy     987.393889   989.77     18  44.833333  65.0     18   
Cloudy                   986.377744   998.99  11782   8.212952  31.0  11782   
Cloudy / Windy           982.557273   986.48     11  40.181818  48.0     11   
Fair                     987.042441  1002.94  47140   8.273271  31.0  47140   
Fair / Windy             987.012113  1002.28    213  37.741784  65.0    213   
Fog                      988.681303   997.02    844   4.944313  22.0    844   
Haze                     987.029688   994.71   2337   6.736842  30.0   2337   
Haze / Windy             975.445000   975.61      2  39.000000  43.0      2   
Heavy Rain               983.480356   992.41    225  16.195556  31.0    225   
Heavy Rain / Windy       979.427059   985.82     17  39.823529  46.0     17   
Heavy T-Storm            985.558000   986.81      5  17.000000  30.0      5   
Light Rain               985.959632   999.65   2038  12.351816  31.0   2038   
Light Rain / Windy       979.701250   987.14     24  36.333333  48.0     24   
Light Rain with Thunder  986.258333   988.78      6  10.500000  24.0      6   
Mostly Cloudy            986.632575   997.67   4664   8.016081  31.0   4664   
Mostly Cloudy / Windy    982.292143   987.47     14  40.500000  57.0     14   
Partly Cloudy            987.138357   999.32   6355   7.972935  31.0   6355   
Partly Cloudy / Windy    984.146800  1001.62     25  3

In [46]:
metro_groups_agg_start = clean_metro_set.groupby("start_station_id").agg(['mean', 'max', 'count'])
metro_groups_agg_start

C:\Users\yusuf\AppData\Local\Temp\ipykernel_5384\1578225388.py:1: FutureWarning: ['start_time', 'end_time', 'bike_id'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  metro_groups_agg_start = clean_metro_set.groupby("start_station_id").agg(['mean', 'max', 'count'])


start_station_lat                   start_station_lon  \
                              mean        max  count              mean   
start_station_id                                                         
3000                     34.025890  34.025890      2       -118.238220   
3005                     34.048501  34.048550  57496       -118.258548   
3006                     34.045540  34.045540  25942       -118.256668   
3007                     34.050480  34.050480  20944       -118.254593   
3008                     34.046612  34.046612  18008       -118.262733   
...                            ...        ...    ...               ...   
4631                     34.085770  34.085770      4       -118.382607   
4632                     33.986671  33.986671    667       -118.440948   
4633                     34.069981  34.069981    100       -118.260094   
4634                     34.047390  34.047390    110       -118.219589   
4638                     34.004650  34.004650     65       -118.255928   

                                    end_station_id               \
                         max  count           mean   max  count   
start_station_id                                                  
3000             -118.238220      2    3027.000000  3054      2   
3005             -118.258530  57496    3197.358842  4634  57496   
3006             -118.256660  25942    3164.346465  4587  25942   
3007             -118.254590  20944    3165.227034  4627  20944   
3008             -118.262730  18008    3222.440082  4627  18008   
...                      ...    ...            ...   ...    ...   
4631             -118.382607      4    4089.250000  4525      4   
4632             -118.440948    667    4480.161919  4632    667   
4633             -118.260094    100    3861.610000  4636    100   
4634             -118.219589    110    3877.936364  4636    110   
4638             -118.255928     65    4430.046154  4638     65   

                 end_station_lat                   end_station_lon  \
                            mean        max  count            mean   
start_station_id                                                     
3000                   34.032555  34.039219      2     -118.237354   
3005                   34.046648  34.156181  57496     -118.255711   
3006                   34.046349  34.145691  25942     -118.255120   
3007                   34.047912  34.177650  20944     -118.254996   
3008                   34.046341  34.154270  18008     -118.258573   
...                          ...        ...    ...             ...   
4631                   34.085927  34.100380      4     -118.296081   
4632                   33.988754  34.094372    667     -118.442688   
4633                   34.058931  34.094929    100     -118.250369   
4634                   34.051260  34.090279    110     -118.239104   
4638                   33.996084  34.046131     65     -118.269961   

                                     
                         max  count  
start_station_id                     
3000             -118.236488      2  
3005             -118.160170  57496  
3006             -118.148230  25942  
3007             -118.225410  20944  
3008             -118.142151  18008  
...                      ...    ...  
4631             -118.261169      4  
4632             -118.291718    667  
4633             -118.219589    100  
4634             -118.219589    110  
4638             -118.248238     65  

[392 rows x 15 columns]

In [47]:
metro_groups_agg_end = clean_metro_set.groupby("end_station_id").agg(['mean', 'max', 'count'])
metro_groups_agg_end

C:\Users\yusuf\AppData\Local\Temp\ipykernel_5384\1209517537.py:1: FutureWarning: ['start_time', 'end_time', 'bike_id'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  metro_groups_agg_end = clean_metro_set.groupby("end_station_id").agg(['mean', 'max', 'count'])


start_station_id              start_station_lat             \
                           mean   max  count              mean        max   
end_station_id                                                              
3000                3044.181818  3082     11         34.044029  34.056610   
3005                3206.157391  4634  61992         34.047711  34.165218   
3006                3150.935031  4587  26428         34.046050  34.147812   
3007                3201.279150  4578  17324         34.049268  34.110031   
3008                3211.372624  4627  18571         34.046390  34.149712   
...                         ...   ...    ...               ...        ...   
4635                4511.000000  4511      1         34.056278  34.056278   
4636                4320.333333  4634      6         34.063447  34.082520   
4638                4448.142857  4638     49         33.998086  34.051102   
4639                4437.000000  4454      3         33.993052  34.017899   
4640                4088.400000  4552     10         34.016294  34.085770   

                      start_station_lon                    end_station_lat  \
                count              mean         max  count            mean   
end_station_id                                                               
3000               11       -118.241302 -118.231277     11       34.025890   
3005            61992       -118.255778 -118.145042  61992       34.048501   
3006            26428       -118.255310 -118.132210  26428       34.045540   
3007            17324       -118.253995 -118.225410  17324       34.050480   
3008            18571       -118.258398 -118.160170  18571       34.046612   
...               ...               ...         ...    ...             ...   
4635                1       -118.231773 -118.231773      1       34.025791   
4636                6       -118.248828 -118.219589      6       34.025742   
4638               49       -118.266307 -118.248238     49       34.004650   
4639                3       -118.277225 -118.248238      3       33.943359   
4640               10       -118.323093 -118.238320     10      -90.000000   

                                 end_station_lon                     
                      max  count            mean         max  count  
end_station_id                                                       
3000            34.025890     11     -118.238220 -118.238220     11  
3005            34.048550  61992     -118.258550 -118.256668  61992  
3006            34.045540  26428     -118.256675 -118.243530  26428  
3007            34.050480  17324     -118.254593 -118.254590  17324  
3008            34.046612  18571     -118.262733 -118.262730  18571  
...                   ...    ...             ...         ...    ...  
4635            34.025791      1     -118.238068 -118.238068      1  
4636            34.025742      6     -118.238068 -118.238068      6  
4638            34.004650     49     -118.255928 -118.255928     49  
4639            33.943359      3     -118.248238 -118.248238      3  
4640           -90.000000     10        0.000000    0.000000     10  

[401 rows x 15 columns]

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
sns.pairplot(clean_weather_set, palette="magma", height=3, hue="cloud_cover_description")

<IPython.core.display.Javascript object>